<a href="https://colab.research.google.com/github/giagia96/NLP---Assignement-4/blob/master/SQUAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tokenizers
!pip install transformers

     |████████████████████████████████| 3.2MB 5.3MB/s 
     |████████████████████████████████| 1.8MB 6.4MB/s 
     |████████████████████████████████| 890kB 19.0MB/s 
     |████████████████████████████████| 2.9MB 18.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=e46b17885d93f873d1c2b1e0ef74468fa13ff4012491dbb5d48e8c43e6540dd7
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: tokenizers 0.10.0
    Uninstalling tokenizers-0.10.0:
      Successfully uninstalled tokenizers-0.10.0


In [2]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig
import argparse

max_len = 384
path = '/content/drive/MyDrive/SQUAD'

with open(path+'/training_set.json') as f:
    raw_train_data = json.load(f)
# Save the slow pretrained tokenizer
slow_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
save_path = "bert_base_uncased/"
if not os.path.exists(save_path):
    os.makedirs(save_path)
slow_tokenizer.save_pretrained(save_path)

# Load the fast tokenizer from saved file
tokenizer = BertWordPieceTokenizer("bert_base_uncased/vocab.txt", lowercase=True)

In [9]:
class SquadExample:
    def __init__(self, question, context, start_char_idx, answer_text, all_answers, qid):
        self.question = question
        self.context = context
        self.start_char_idx = start_char_idx
        self.answer_text = answer_text
        self.all_answers = all_answers
        self.qid = qid
        self.skip = False

    def preprocess(self):
        context = self.context
        question = self.question
        answer_text = self.answer_text
        start_char_idx = self.start_char_idx

        # Clean context, answer and question
        context = " ".join(str(context).split())
        question = " ".join(str(question).split())
        answer = " ".join(str(answer_text).split())

        # Find end character index of answer in context
        end_char_idx = start_char_idx + len(answer)
        if end_char_idx >= len(context):
            self.skip = True
            return

        # Mark the character indexes in context that are in answer
        is_char_in_ans = [0] * len(context)
        for idx in range(start_char_idx, end_char_idx):
            is_char_in_ans[idx] = 1

        # Tokenize context
        tokenized_context = tokenizer.encode(context)

        # Find tokens that were created from answer characters
        ans_token_idx = []
        for idx, (start, end) in enumerate(tokenized_context.offsets):
            if sum(is_char_in_ans[start:end]) > 0:
                ans_token_idx.append(idx)

        if len(ans_token_idx) == 0:
            self.skip = True
            return

        # Find start and end token index for tokens from answer
        start_token_idx = ans_token_idx[0]
        end_token_idx = ans_token_idx[-1]

        # Tokenize question
        tokenized_question = tokenizer.encode(question)

        # Create inputs
        input_ids = tokenized_context.ids + tokenized_question.ids[1:]
        token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(
            tokenized_question.ids[1:]
        )
        attention_mask = [1] * len(input_ids)

        # Pad and create attention masks.
        # Skip if truncation is needed
        padding_length = max_len - len(input_ids)
        if padding_length > 0:  # pad
            input_ids = input_ids + ([0] * padding_length)
            attention_mask = attention_mask + ([0] * padding_length)
            token_type_ids = token_type_ids + ([0] * padding_length)
        elif padding_length < 0:  # skip
            self.skip = True
            return

        self.input_ids = input_ids
        self.token_type_ids = token_type_ids
        self.attention_mask = attention_mask
        self.start_token_idx = start_token_idx
        self.end_token_idx = end_token_idx
        self.context_token_to_char = tokenized_context.offsets

def create_squad_examples(raw_data):
    squad_examples = []
    for item in raw_data["data"]:
        for para in item["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                question = qa["question"]
                qid = qa["id"]
                answer_text = qa["answers"][0]["text"]
                all_answers = [_["text"] for _ in qa["answers"]]
                start_char_idx = qa["answers"][0]["answer_start"]
                squad_eg = SquadExample(
                    question, context, start_char_idx, answer_text, all_answers, qid
                )
                squad_eg.preprocess()
                squad_examples.append(squad_eg)
    return squad_examples


def create_inputs_targets(squad_examples):
    dataset_dict = {
        "input_ids": [],
        "token_type_ids": [],
        "attention_mask": [],
        "start_token_idx": [],
        "end_token_idx": [],
    }
    for item in squad_examples:
        if item.skip == False:
            for key in dataset_dict:
                dataset_dict[key].append(getattr(item, key))
    for key in dataset_dict:
        dataset_dict[key] = np.array(dataset_dict[key])

    x = [
        dataset_dict["input_ids"],
        dataset_dict["token_type_ids"],
        dataset_dict["attention_mask"],
    ]
    y = [dataset_dict["start_token_idx"], dataset_dict["end_token_idx"]]
    return x, y


train_squad_examples1 = create_squad_examples(raw_train_data)
train_squad_examples = train_squad_examples1[101:]
x_train, y_train = create_inputs_targets(train_squad_examples)
print(f"{len(train_squad_examples)} training points created.")
eval_squad_examples = train_squad_examples1[:100]
x_eval, y_eval = create_inputs_targets(eval_squad_examples)
print(f"{len(eval_squad_examples)} evaluation points created.")

87498 training points created.
100 evaluation points created.


Create the Question-Answering Model using BERT and Functional API


In [18]:
def create_model():
    ## BERT encoder
    encoder = TFBertModel.from_pretrained("bert-base-uncased")
    encoder.config.max_position_embeddings = max_len
    ## QA Model
    input_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    token_type_ids = layers.Input(shape=(max_len,), dtype=tf.int32)
    attention_mask = layers.Input(shape=(max_len,), dtype=tf.int32)
    embedding = encoder.bert(
        input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask
    )[0]

    start_logits = layers.Dense(1, name="start_logit", use_bias=False)(embedding)
    start_logits = layers.Flatten()(start_logits)

    end_logits = layers.Dense(1, name="end_logit", use_bias=False)(embedding)
    end_logits = layers.Flatten()(end_logits)

    start_probs = layers.Activation(keras.activations.softmax)(start_logits)
    end_probs = layers.Activation(keras.activations.softmax)(end_logits)

    model = keras.Model(
        inputs=[input_ids, token_type_ids, attention_mask],
        outputs=[start_probs, end_probs],
    )
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    optimizer = keras.optimizers.Adam(lr=5e-5)
    model.compile(optimizer=optimizer, loss=[loss, loss])
    return model


In [19]:
def normalize_text(text):
    text = text.lower()

    # Remove punctuations
    exclude = set(string.punctuation)
    text = "".join(ch for ch in text if ch not in exclude)

    # Remove articles
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    text = re.sub(regex, " ", text)

    # Remove extra white space
    text = " ".join(text.split())
    return text

class ExactMatch_F1(keras.callbacks.Callback):
    def __init__(self, x_eval, y_eval):
        self.x_eval = x_eval
        self.y_eval = y_eval

    def on_epoch_end(self, epoch, logs=None):
        pred_start, pred_end = self.model.predict(self.x_eval)
        count = 0
        eval_examples_no_skip = [_ for _ in eval_squad_examples if _.skip == False]
        for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
            squad_eg = eval_examples_no_skip[idx]
            offsets = squad_eg.context_token_to_char
            start = np.argmax(start)
            end = np.argmax(end)
            if start >= len(offsets):
                continue
            pred_char_start = offsets[start][0]
            if end < len(offsets):
                pred_char_end = offsets[end][1]
                pred_ans = squad_eg.context[pred_char_start:pred_char_end]
            else:
                pred_ans = squad_eg.context[pred_char_start:]
            normalized_pred_ans = normalize_text(pred_ans)
            normalized_true_ans = [normalize_text(_) for _ in squad_eg.all_answers]
            if normalized_pred_ans in normalized_true_ans:
                count += 1
        acc = count / len(self.y_eval[0])
        print(f"\nepoch={epoch+1}, exact match score={acc:.2f}")

In [20]:
def train():
  try:
    # Create distribution strategy
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)

    # Create model
    with strategy.scope():
        model = create_model()
  except:
      model = create_model()

  model.summary()
  exact_match_callback = ExactMatch_F1(x_eval, y_eval)
  model.fit(
      x_train,
      y_train,
      epochs=15,  # For demonstration, 3 epochs are recommended
      batch_size=256,
      callbacks=[exact_match_callback],
  )
  model.save_weights(path+'/model.h5')
  return model

model = train()

INFO:tensorflow:Initializing the TPU system: grpc://10.97.208.50:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.97.208.50:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)
Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_12 (InputLayer)           [(None, 384)]        0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           [(None, 384)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 109482240   input_10[0][0]                   
                                                                 input_12[0][0]             

  6/337 [..............................] - ETA: 3:17 - loss: 10.9105 - activation_10_loss: 5.2952 - activation_11_loss: 5.6153WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0037s vs `on_train_batch_end` time: 8.9123s). Check your callbacks.


337/337 [==============================] - 315s 715ms/step - loss: 4.1794 - activation_10_loss: 2.1282 - activation_11_loss: 2.0513

epoch=1, exact match score=0.67
Epoch 2/15
337/337 [==============================] - 200s 594ms/step - loss: 1.7975 - activation_10_loss: 0.9520 - activation_11_loss: 0.8455

epoch=2, exact match score=0.71
Epoch 3/15
337/337 [==============================] - 200s 595ms/step - loss: 1.3284 - activation_10_loss: 0.7076 - activation_11_loss: 0.6207

epoch=3, exact match score=0.75
Epoch 4/15
337/337 [==============================] - 200s 595ms/step - loss: 0.9932 - activation_10_loss: 0.5356 - activation_11_loss: 0.4577

epoch=4, exact match score=0.71
Epoch 5/15
337/337 [==============================] - 200s 595ms/step - loss: 0.7602 - activation_10_loss: 0.4144 - activation_11_loss: 0.3458

epoch=5, exact match score=0.74
Epoch 6/15
337/337 [==============================] - 200s 595ms/step - loss: 0.5621 - activation_10_loss: 0.3055 - activation_11_l

In [21]:

def dataset_from_file(filename):
  with open(filename) as f:
    raw_data = json.load(f) 
  squad_examples = create_squad_examples(raw_train_data, True)
  print(f"{len(train_squad_examples)} points created.")

  return squad_examples

def __main__():
  parser = argparse.ArgumentParser('')
  parser.add_argument('data_file', metavar='data.json', help='Input data JSON file.')
  args =  parser.parse_args()
  squad_examples = dataset_from_file(args.data_file)
  model = create_model()
  model.load_weights(path+'/model.h5')
  pred_dict = {}
  for ex in squad_examples:
    x, y = create_inputs_targets(ex)
    pred_start, pred_end = model.predict(x)
    offsets = ex.context_token_to_char
    start = np.argmax(pred_start)
    end = np.argmax(pred_end)
    if start >= len(offsets):
       continue
    pred_char_start = offsets[start][0]
    if end < len(offsets):
       pred_char_end = offsets[end][1]
       pred_ans = squad_eg.context[pred_char_start:pred_char_end]
    else:
       pred_ans = squad_eg.context[pred_char_start:]
    pred_dict[ex.qid] = normalize_text(pred_ans)
  with open('predictions.txt', 'w') as file:
     file.write(json.dumps(pred_dict)) 
  